In [40]:
import numpy as np
import numpy.linalg as npl
import scipy.sparse as spsp

In [37]:
def training_set_creator(*args):
    """
    args are a non-zero number of lists of size 3.
    Example of use: we want the creation of an iteratable representing all possible elements on a n-dimensional grid
    first dimension goes from a to b with c elements, then you pass "[a,b,c]" as a first argument.
    return: n-dimensional grid on which you can iterate.
    
    """
    linspace_args = (np.linspace(arg[0],arg[1],arg[2]) for arg in args)
    
    meshes = np.meshgrid(*linspace_args)
    dimensions = [mesh.ravel() for mesh in meshes]
    
    tuples = zip(*dimensions)
#     return tuples
    for tup in tuples:
        yield(tuple(tup))

In [38]:
def apply_mu(mu, A_q, shape):
    """
    from output of pre_computer and a mu: computes A_rb and f_rb and u_rb, the solution of A_rb.u_rb=f_rb
    """
    A = A_q[1]
    f = A_q[3]
    
    for i in range(len(mu) - shape):
        A += A_q[0][i] * mu[i]
        f += A_q[2][i] * mu[i]
    
    return A, f

In [55]:
n = 2
N_diff = 2**n
f_diff = np.zeros(N_diff+2)
f_diff[0] = 1

#construction de la matrice A en format sparse
tab_A_0 = [np.repeat([1,0], [N_diff//2,N_diff//2], 0),
           np.repeat([0,-2,-1,0], [1,N_diff//2-1,1,N_diff//2], 0),
           np.repeat([0,1,0], [1,N_diff//2-1,N_diff//2], 0)]
tab_A_1 = [np.repeat([0,((N_diff+2)**2)], [N_diff//2,N_diff//2],0),
           np.repeat([1,0,-((N_diff+2)**2),-2*((N_diff+2)**2)],[1,N_diff//2-1,1,N_diff//2],0),
           np.repeat([0,((N_diff+2)**2)],[N_diff//2,N_diff//2],0)]
A_0 = sp.sparse.diags(tab_A_0,[-1,0,1],(N_diff+1,N_diff+1)) * ((N_diff+2)**2)
A_1 = sp.sparse.diags(tab_A_1,[-1,0,1],(N_diff+1,N_diff+1))


In [57]:
A_0.todense(), A_1.todense()

(matrix([[  0.,   0.,   0.,   0.,   0.],
         [ 36., -72.,  36.,   0.,   0.],
         [  0.,  36., -36.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]]),
 matrix([[  1.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0., -36.,  36.,   0.],
         [  0.,   0.,  36., -72.,  36.],
         [  0.,   0.,   0.,  36., -72.]]))

In [32]:
#résolution par diff finies
def pre_computer(Base):
    """
    pre_computer take as input a reduced_base and pre_computes all quantities used for a reduced_solver.
    output: [[A_rb^q](0<= i <=n1),A_rb^0,[f_rb^q](0<= i <=n2),f_rb^0],n
    with A_rb = sum_(i in 0..n1){ mu[i]*A_rb^(q_1)_i } + A_rb^0
    and  f_rb = sum_(i in n1+1..n2){ mu[i]*f_rb^(q_1)_i } + f_rb^0
            n = n1+1
    """
    
    #A = A_1 + mu[0]*A_0
    #f = mu[1] * f_diff
    
    return [[Base.T @ A_0.todense() @ Base],
            Base.T @ A_1.todense() @ Base ,
            [Base.T @ f_diff],
            Base.T @ np.zeros_like(f_diff)], 1

In [2]:
def greedy_algorithm(tol, MUs, A_delta, f_delta, n=1):
    # On prend un mu (au hasard) pour initialiser l'algorithme
    mu_1 = MUs[0]
    u_delta_mu_1 = npl.solve(A_delta(mu_1), f_delta(mu_1))
    np.delete(MUs, 0)
    
    # B est la matrice de changement de base de A_delta à A_rb
    B = np.copy(u_delta_mu_1)
    
    # A @ u = f pour un mu donné
    
    # On calcule tous les u_delta pour éviter de le refair dans la boucle par la suite
    U_DELTA = np.array([npl.solve(A_delta(mu), f_delta(mu)) for mu in MUs])
    
    err = 1e10
    
    while err > tol and MUs.shape[0] > 0:
        
        eta = np.zeros(MUs.shape[0])
        
        for idx_mu, mu in enumerate(MUs):
            A_delta_mu = A_delta(mu)
            f_delta_mu = f_delta(mu)
        
            u_delta_mu = U_DELTA[idx_mu]
            
            # A_rb_mu @ u_rb_mu = f_rb_mu
            # A_rb_mu = B.T @ A_delta_mu @ B
            u_rb_mu = npl.solve(B.T @ A_delta_mu @ B, B.T @ f_delta_mu)
            
            # Pour l'instant, eta(mu) = ||u_delta - u_rb||_L2
            eta[idx_mu] = npl.norm(u_delta_mu - u_rb_mu, ord=2)
        
        # On cherche la pire approximation
        idx_worst_mu = np.argmax(eta)
        
        B = np.vstack((B, U_DELTA[idx_worst_mu]))
        
        np.delete(MUs[idx_worst_mu])
        np.delete(U_DELTA[idx_worst_mu])
        
        err = eta[idx_worst_mu]
        
    return B